In [1]:
import numpy as np
import pandas as pd

In [2]:
path = 'from_104_to_128.pk'

data = pd.read_pickle(path)
data

260        101110111001110110011111
380        110000001110101111101000
388        001011101000100010110000
428        101110100000101100000100
635        010111111010101010101111
                     ...           
3373667                    00000000
3373668                    00000000
3373669                    00000000
3373670                    00000000
3373671                    00000000
Name: ip, Length: 1994891, dtype: object

In [3]:
# test
tail_from_104 = '101110111001110110011111'
answer = '240e:e2:1c:2f99:e6bd:4bde:b4bb:9d9f'

from_104_to_120 = tail_from_104[:16]
tail = tail_from_104[16:]

bin_to_dml_1 = int(from_104_to_120, 2)
hex1 = hex(bin_to_dml_1)

bin_to_dml_2 = int(tail, 2)
hex2 = hex(bin_to_dml_2)

print(hex1[2:], hex2[2:])

bb9d 9f


In [4]:
%%markdown
显然进制转换是正确的哈

显然进制转换是正确的哈


In [5]:
def format(arg):
    x = len(arg)
    if x < 24:
        arg = '0'*(24 - x) + arg
    return arg

In [6]:
len(format('00000000'))

24

In [7]:
type(data)

pandas.core.series.Series

In [8]:
data = data.apply(format)

In [9]:
# 获得原数据的两个副本用来进行不同段的取值
series_1 = data.copy()
series_2 = data.copy()

In [10]:
series_1 = series_1.str.slice(start=0, stop=16)

In [11]:
series_2 = series_2.str.slice(start=16)

In [12]:
series_1

260        1011101110011101
380        1100000011101011
388        0010111010001000
428        1011101000001011
635        0101111110101010
                 ...       
3373667    0000000000000000
3373668    0000000000000000
3373669    0000000000000000
3373670    0000000000000000
3373671    0000000000000000
Name: ip, Length: 1994891, dtype: object

In [13]:
series_2

260        10011111
380        11101000
388        10110000
428        00000100
635        10101111
             ...   
3373667    00000000
3373668    00000000
3373669    00000000
3373670    00000000
3373671    00000000
Name: ip, Length: 1994891, dtype: object

In [14]:
%%time
series_1.value_counts() / series_1.count()

Wall time: 312 ms


0000000000000000    5.287918e-01
0000000000000001    3.374821e-02
0000000000000010    1.824811e-02
0000000000100000    7.473591e-03
0000000100000000    6.984843e-03
                        ...     
1010011011001011    5.012805e-07
0101001101001101    5.012805e-07
1100010101101101    5.012805e-07
1010000000010001    5.012805e-07
1110011000111010    5.012805e-07
Name: ip, Length: 43949, dtype: float64

In [15]:
series_2.value_counts() / series_2.count()

00000001    0.230085
00000010    0.075670
00000000    0.062236
00000011    0.016777
10100001    0.011658
              ...   
11011011    0.000908
11111000    0.000908
11011100    0.000893
11011000    0.000893
11101100    0.000835
Name: ip, Length: 256, dtype: float64

In [16]:
# index_1 = series_1.value_counts().index

In [17]:
# index_2 = series_2.value_counts().index

In [18]:
def bin_to_hex(string):
    demical = int(string, 2)
    return hex(demical)[2:]

test = '10011111'
test_1 = '1011101110011101'
bin_to_hex(test_1)

'bb9d'

In [19]:
# memory_list_1 = []

# for index in index_1:
#     ret = bin_to_hex(index)
#     memory_list_1.append(ret)

# memory_list_2 = []   

# for index in index_2:
#     ret = bin_to_hex(index)
#     memory_list_2.append(ret)

# list(zip(memory_list_1, memory_list_2))

In [20]:
series_1 = series_1.apply(bin_to_hex)
series_2 = series_2.apply(bin_to_hex)

In [21]:
series_1.value_counts() / series_1.count()

0       5.287918e-01
1       3.374821e-02
2       1.824811e-02
20      7.473591e-03
100     6.984843e-03
            ...     
3b34    5.012805e-07
88fa    5.012805e-07
d0cd    5.012805e-07
3591    5.012805e-07
9dce    5.012805e-07
Name: ip, Length: 43949, dtype: float64

In [22]:
series_2.value_counts() / series_2.count()

1     0.230085
2     0.075670
0     0.062236
3     0.016777
a1    0.011658
        ...   
db    0.000908
f8    0.000908
d8    0.000893
dc    0.000893
ec    0.000835
Name: ip, Length: 256, dtype: float64

In [23]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

# te = TransactionEncoder()
# df_tf = te.fit_transform()

因为我们的目的是要找到104-120 和 120-128 之间的关联，而频繁项集的查找算是找到数据集的所有特征的关联规则
为了区分最后得到的关联规则是 104-120 -> 120-128  or 120-128 -> 104-120 的，我们将后104~120的十六进制全部补齐为两位
那么最后只要筛选**规则前导项==2**和**规则后导项==2**，即可分辨

In [24]:
# 补齐series_2 中全为 2 位
# 补齐series_1 中全为 4 位

In [25]:
series_1

260        bb9d
380        c0eb
388        2e88
428        ba0b
635        5faa
           ... 
3373667       0
3373668       0
3373669       0
3373670       0
3373671       0
Name: ip, Length: 1994891, dtype: object

In [26]:
def format_series_1(arg):
    l = len(arg)
    if l < 4:
        return '0'*(4 - l) + arg
    return arg

series_1.apply(format_series_1)

260        bb9d
380        c0eb
388        2e88
428        ba0b
635        5faa
           ... 
3373667    0000
3373668    0000
3373669    0000
3373670    0000
3373671    0000
Name: ip, Length: 1994891, dtype: object

In [27]:
series_1 = series_1.apply(format_series_1)

In [28]:
series_2

260        9f
380        e8
388        b0
428         4
635        af
           ..
3373667     0
3373668     0
3373669     0
3373670     0
3373671     0
Name: ip, Length: 1994891, dtype: object

In [29]:
def format_series_2(arg):
    if len(arg) == 1:
        return '0' + arg
    return arg

series_2.apply(format_series_2)

260        9f
380        e8
388        b0
428        04
635        af
           ..
3373667    00
3373668    00
3373669    00
3373670    00
3373671    00
Name: ip, Length: 1994891, dtype: object

In [30]:
series_2 = series_2.apply(format_series_2)

In [31]:
df = pd.DataFrame(list(zip(series_1, series_2)), columns=['104-120', '120-128'])
df

,104-120,120-128
0,bb9d,9f
1,c0eb,e8
2,2e88,b0
3,ba0b,04
4,5faa,af
5,5f06,ff
6,567c,95
7,5c25,7d
8,bb3b,77
9,bb4b,25


In [32]:
%%time
te = TransactionEncoder()
df_tf = te.fit_transform(df)

Wall time: 0 ns


In [33]:
df_tf

array([[ True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True, False,  True],
       [False, False, False, False, False, False],
       ...,
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False]])

In [34]:
%%time
# df_arr = df.stack().groupby(level=0).apply(list).tolist()
df.values

Wall time: 0 ns


array([['bb9d', '9f'],
       ['c0eb', 'e8'],
       ['2e88', 'b0'],
       ...,
       ['0000', '00'],
       ['0000', '00'],
       ['0000', '00']], dtype=object)

In [35]:
%%markdown
当直接使用TransactionEncoder的时候，np.zeros()由于无法生成过大维度的数组会报错：
Unable to allocate array with shape (1994891, 44205) and data type bool

44205 = 104-120 的 unique值 + 120-128的 unique值，所以为了使用TransactionEncoder，要先除去support很小的值

当直接使用TransactionEncoder的时候，np.zeros()由于无法生成过大维度的数组会报错：
Unable to allocate array with shape (1994891, 44205) and data type bool

44205 = 104-120 的 unique值 + 120-128的 unique值，所以为了使用TransactionEncoder，要先除去support很小的值


In [36]:
np.zeros((100000, 1000), dtype=bool)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [37]:
np.zeros((2000000, 300), dtype=bool)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [38]:
# series_1[series_1.value_counts() > 1] 这样索引出数量大于 1 的东西是不行的，因为不是使用同一组索引index

series_1.value_counts()[series_1.value_counts() > 1000]

0000    1054882
0001      67324
0002      36403
0020      14909
0100      13934
         ...   
1900       1109
0023       1052
001f       1032
0024       1009
3500       1004
Name: ip, Length: 82, dtype: int64

In [39]:
# 筛选出出现次数 > 1000的 104-120，剩下 82个了

indices = _.index

In [40]:
indices

Index(['0000', '0001', '0002', '0020', '0100', '0003', '0010', '0093', '0004',
       '0005', '0200', '0006', '0007', '0008', '0009', '000a', '0011', '0300',
       '00ff', '0600', '000c', '0700', '000d', '4093', '000b', 'c093', '000f',
       '8000', '0013', '0101', 'ffff', '000e', '8093', 'ff00', '330a', '0400',
       '0012', '0500', '1000', '4000', '0015', '0017', '1100', '0800', '0018',
       '0016', '0014', '1200', '0900', '0201', '0019', '2000', '1400', '0a00',
       '001c', '001e', '2100', '1300', '001a', '1600', '0030', '0102', '0022',
       '5300', '001b', '0f00', '1800', '2800', 'c000', '1500', '0301', '7000',
       '3000', '0026', '001d', '0021', '1700', '1900', '0023', '001f', '0024',
       '3500'],
      dtype='object')

In [41]:
series_2.value_counts()

01    458995
02    150953
00    124155
03     33469
a1     23257
       ...  
db      1812
f8      1812
d8      1781
dc      1781
ec      1665
Name: ip, Length: 256, dtype: int64

In [42]:
series_1.values

array(['bb9d', 'c0eb', '2e88', ..., '0000', '0000', '0000'], dtype=object)

In [43]:
df

,104-120,120-128
0,bb9d,9f
1,c0eb,e8
2,2e88,b0
3,ba0b,04
4,5faa,af
5,5f06,ff
6,567c,95
7,5c25,7d
8,bb3b,77
9,bb4b,25


In [44]:
# 找出次数 >1000 的对应的 df 切片
def filter_value(value):
    return value if value in indices else 'verbose'

# 只需要筛选出 104-120这部分符合条件的即可

slice_df = df['104-120'].apply(filter_value)

In [45]:
slice_df

0          verbose
1          verbose
2          verbose
3          verbose
4          verbose
            ...   
1994886       0000
1994887       0000
1994888       0000
1994889       0000
1994890       0000
Name: 104-120, Length: 1994891, dtype: object

In [46]:
slice_df[slice_df != 'verbose']

20         0000
21         0000
22         0000
23         0000
24         0000
           ... 
1994886    0000
1994887    0000
1994888    0000
1994889    0000
1994890    0000
Name: 104-120, Length: 1394674, dtype: object

In [47]:
slice_df[slice_df != 'verbose'].value_counts()

0000    1054882
0001      67324
0002      36403
0020      14909
0100      13934
         ...   
1900       1109
0023       1052
001f       1032
0024       1009
3500       1004
Name: 104-120, Length: 82, dtype: int64

In [48]:
df_ = df.copy()

In [49]:
df_['104-120'] = slice_df[slice_df != 'verbose']

In [50]:
df_['104-120'][df_['104-120'].isnull()]

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1994236    NaN
1994462    NaN
1994628    NaN
1994856    NaN
1994863    NaN
Name: 104-120, Length: 600217, dtype: object

In [51]:
df_.dropna().shape

(1394674, 2)

现在这个维度显然是正确的了，这样拿去np.zeros() 的维度是 1394674 * 82+256 维，应该没问题，在老师的linux系统上面跑的话，numpy内存问题不会那么严重，我这个win好像8太行

In [52]:
df_.dropna(inplace=True)

In [53]:
df_

,104-120,120-128
20,0000,0f
21,0000,0d
22,0000,0c
23,0000,0a
24,0000,09
25,0000,07
26,0000,06
27,0000,05
28,0000,04
29,0000,03


In [54]:
df_.dropna(inplace=True)

In [55]:
df_.shape

(1394674, 2)

In [56]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
df_tf = te.fit_transform(df_.values)

In [57]:
df_tf

array([[False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False]])

In [58]:
df_tf.shape

(1394674, 338)

正确的编码维度 256 + 82

In [59]:
df = pd.DataFrame(df_tf, columns=te.columns_)

In [60]:
df

,00,0000,0001,0002,0003,0004,0005,0006,0007,0008,...,f8,f9,fa,fb,fc,fd,fe,ff,ff00,ffff
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### **找出频繁项集**

In [61]:
from mlxtend.frequent_patterns import fpgrowth, apriori

help(fpgrowth)

Help on function fpgrowth in module mlxtend.frequent_patterns.fpgrowth:

fpgrowth(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0)
    Get frequent itemsets from a one-hot DataFrame
    
    Parameters
    -----------
    df : pandas DataFrame or pandas SparseDataFrame
      pandas DataFrame the encoded format.
      The allowed values are either 0/1 or True/False.
      For example,
    
    ```
             Apple  Bananas  Beer  Chicken  Milk  Rice
        0      1        0     1        1     0     1
        1      1        0     1        0     0     1
        2      1        0     1        0     0     0
        3      1        1     0        0     0     0
        4      0        0     1        1     1     1
        5      0        0     1        0     1     1
        6      0        0     1        0     1     0
        7      1        1     0        0     0     0
    ```
    
    min_support : float (default: 0.5)
      A float between 0 and 1 for minimum support of

In [70]:
%%time

frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)

Wall time: 3.66 s


In [72]:
frequent_itemsets.sort_values(by='support',ascending=False,inplace=True)

In [73]:
frequent_itemsets

,support,itemsets
1,0.756365,(0000)
2,0.288662,(01)
5,0.273384,"(01, 0000)"
3,0.100417,(02)
6,0.092804,"(02, 0000)"
0,0.084228,(00)
4,0.078973,"(00, 0000)"


### **生成关联规则**

In [76]:
from mlxtend.frequent_patterns import association_rules
 
association_rule = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.7)	# metric可以有很多的度量选项，返回的表列名都可以作为参数
association_rule.sort_values(by='leverage',ascending=False,inplace=True)    #关联规则可以按leverage排序

In [77]:
association_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(01),(0000),0.288662,0.756365,0.273384,0.947073,1.252138,0.055050,4.603198
1,(02),(0000),0.100417,0.756365,0.092804,0.924191,1.221885,0.016853,3.213798
2,(00),(0000),0.084228,0.756365,0.078973,0.937618,1.239638,0.015267,3.905548


根据置信度可以看出，在最后八位选定为01、02、00的情况下，90%以上的人会把104-120位定为0000